In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd

import PIL
from PIL import ImageFont, ImageDraw, Image

import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.keras.models import load_model
from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes,read_classes, read_anchors, preprocess_image

%matplotlib inline

In [ ]:
def yolo_filter_box(boxes, box_confidence, box_class_probs, threshold = 0.6):
    box_scores = box_confidence*box_class_probs #computing box score
    box_classes = tf.math.argmax(box_scores, axis=-1) #index contaning max score
    box_class_scores = tf.math.reduce_max(box_scores,axis=-1) # max score corresponding to the box
    filtering_mask = box_class_scores >=threshold
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [ ]:
def intersection_over_union(box1, box2):
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # calculating the coordinates of the intersection of box1 and box2
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = max(box1[2], box2[2])
    yi2 = max(box1[3], box2[3])
    
    intersection_width = xi2 - xi1
    intersection_height = yi2 - yi1
    
    intersection_area = max(intersection_height,0) * max(intersection_width,0)
    
    #calculationg union aea: Union(box1,box2) = box1 + box2 - intesection(box1,box2)
    
    box1_area = (box1[3] - box1[1]) * (box1[2] - box1[0])
    box2_area = (box2[3] - box2[1]) * (box2[2] - box2[0])
    
    union_area = box1_area + box2_area -intersection_area
    
    iou = intersection_aea / union_area #iou: intersection over union.
    
    return iou
    

In [ ]:
def yolo_non_max_suppressions(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    
    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32') #tensor that is to be used in non_max_supression.
    nms_indices = tf.image.non_max_suppression(boxes,
                                               scores,     
                                               max_boxes,
                                               iou_threshold=0.5,
                                               name=None
                                              )
    scores = tf.gather(scores, nms_indices)
    boxes = tf.gather(boxes, nms_indices)
    classes = tf.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [ ]:
def yolo_boxes_to_corners(box_xy, box_wh):
    #converting yolo boxe predictions to bounding boxes.
    box_mins = box_xy - (box_wh/ 2.)
    box_maxes = box_xy +(box_wh/ 2.)
    
    return tf.keras.backend.concatenate([
        box_mins[...,1:2],  #y_min
        box_mins[...,0:1],  #x_min
        box_maxes[...,1:2], #y_max
        box_maxes[...,0:1]  #x_max
    ])

In [ ]:
def yolo_out(yolo_outputs, image_shape= (720,1280), max_boxes=5, score_threshold=0.6, iou_threshold=0.5):
    
    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs
    boxes = yolo_boxes_to_corners(box_xy,box_wh)
    
    scores,boxes,classes = yolo_filter_box(boxes, box_confidence, box_class_probs, score_threshold)
    
    boxes = scale_boxes(boxes,image_shape)
    
    scores, boxes, classes = yolo_non_max_suppressions(scores, boxes, classes, max_boxes=10, iou_threshold=0.5)
    
    return scores, boxes, classes

In [ ]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
model_image_size = (608, 608) # Same as yolo_model input layer size

In [ ]:
yolo_model = load_model('model_data/', compile=True)

In [ ]:
yolo_model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 608, 608, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 608, 608, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 608, 608, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
def predections(image_file):
    
    image, image_data = preprocess_image('images/' + image_file, model_image_size=(608,608))
    
    yolo_model_outputs = yolo_model(image_data)
    yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))
    
    out_scores, out_boxes, out_classes = yolo_out(yolo_outputs, [image.size[1], image.size[0]], 10,0.3,0.5)
    
    print('Found {} boxes for {}'.format(len(out_boxes), 'images/' + image_file))
    colors = get_colors_for_classes(len(class_names))
    draw_boxes(image, out_boxes, out_classes, class_names, out_scores)
    image.save(os.path.join('C:/Users/91942/PycharmProjects/deepL/deeplTENSORFLOW/object_detection/out',image_file), quality=100)
    
    output_image = Image.open(os.path.join('C:/Users/91942/PycharmProjects/deepL/deeplTENSORFLOW/object_detection/out', image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes


In [ ]:
out_scores, out_boxes, out_classes = predections('test.jpg')